# Размещение баннеров

In [17]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

In [12]:
df = pd.read_csv("checkins.dat", sep = '|', skipinitialspace = True)

In [13]:
df.dropna(inplace = True)
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [40]:
data = df.loc[:,['latitude      ', 'longitude     ']][:100000]
data.shape

(100000, 2)

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [41]:
clustering = MeanShift(bandwidth=0.1).fit(data)

In [43]:
clustering.cluster_centers_.shape

(3231, 2)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [47]:
labels = np.zeros(clustering.cluster_centers_.shape[0])
for i in clustering.labels_:
    labels[i] += 1

In [48]:
labels = np.where(labels>15)

Как мы помним, 20 баннеров надо разместить близ офисов компании.

In [50]:
carnival_office = [[33.751277, -118.188740],
                   [25.867736, -80.324116],
                   [51.503016, -0.075479],
                   [52.378894, 4.885084],
                   [39.366487, 117.036146],
                   [-33.868457, 151.205134]]

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [56]:
big_clusters = clustering.cluster_centers_[labels, :][0]

In [58]:
dist = []
for center in big_clusters:
    x=[]
    for o in carnival_office:
        x.append(sum((center - o)**2))
    dist.append(np.min(np.array(x)))

In [62]:
place = np.argsort(dist)[:20]

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса.

In [63]:
big_clusters[place[0]]

array([-33.86063043, 151.20477593])